In [8]:
import os
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset
import numpy as np

from PIL import Image
from pathlib import Path

# Recycles image classification

In [2]:
%%writefile Dataset_Class.py

## class
import os
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import shutil

class PyTorch_Classification_Dataset_Class(Dataset):
  def __init__(self,
               dataset_dir, # = '/content/Recycle_Classification_Dataset',
               transform):
    super().__init__()

    self.image_abs_path = dataset_dir
    self.transform = transform
    self.label_list = os.listdir(self.image_abs_path) # ['can', 'glass', 'paper', 'plastic']
    self.label_list.sort() # optional
    self.x_list = []
    self.y_list = []
    for label_index, label_str in enumerate(self.label_list):
      img_path = os.path.join(self.image_abs_path, label_str)
      img_list = os.listdir(img_path)
      for img in img_list:
        self.x_list.append(os.path.join(img_path, img))
        self.y_list.append(label_index)

  def __len__(self):
    return len(self.x_list)

  def __getitem__(self, idx):
    image = Image.open(self.x_list[idx])
    if image.mode != 'RGB':
      image = image.convert('RGB')
    image = self.transform(image)
    return image, self.y_list[idx]  # tensor

  def __save_label_map__(self, dst_text_path='label_map.txt'):
    label_list = self.label_list
    f = open(dst_text_path, 'w')
    for i in range(len(label_list)):
      f.write(label_list[i] + '\n')
    f.close()

  def __num_classes__(self):
    return len(self.label_list)

Overwriting Dataset_Class.py


## Model from scratch

In [3]:
%%writefile Model_Class_From_the_Scratch.py

import torch
from torch import nn
import torch.nn.functional as F

class MODEL_From_Scratch(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.classifier = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),  ## Conv layer
        nn.Flatten(),
        nn.Linear(128, 512),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(512, 64),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(64, num_classes)  ## Fully connected layer
    )

  def forward(self, x):
    return self.classifier(x)

Overwriting Model_Class_From_the_Scratch.py


## MobileNet class

In [4]:
%%writefile Model_Class_Transfer_Learning_MobileNet.py

import torch
from torchvision import models
from torch import nn

class MobileNet(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    weights = models.MobileNet_V2_Weights.IMAGENET1K_V2
    self.network = models.mobilenet_v2(weights=weights)
    ## Transfer learning parameter freezing
    for param in self.network.parameters():
      param.requires_grad = False
    in_features = self.network.classifier[1].in_features
    self.network.classifier[1] = nn.Linear(in_features, num_classes)

  def forward(self, x):
    x = self.network(x)
    return x

Overwriting Model_Class_Transfer_Learning_MobileNet.py


In [35]:
weights = models.MobileNet_V2_Weights.IMAGENET1K_V2
weights.transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [22]:
num_classes = 4
weights = models.MobileNet_V2_Weights.IMAGENET1K_V2
model = models.mobilenet_v2(weights=weights)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

## Training class

In [5]:
%%writefile PyTorch_Classification_Training_Class.py

import os
import torch
from torch import optim
from torchvision import transforms
import torch.nn.functional as F
from tqdm import tqdm
import shutil

from Model_Class_From_the_Scratch import MODEL_From_Scratch
from Model_Class_Transfer_Learning_MobileNet import MobileNet
from Dataset_Class import PyTorch_Classification_Dataset_Class as Dataset

class PyTorch_Classification_Training_Class():
  def __init__(self,
               dataset_dir=os.path.join(os.getcwd(), "Recycle_Classification_Dataset"),
               batch_size=16,
               train_ratio = 0.75
               ):
    if not os.path.isdir(dataset_dir):
      os.system('git clone https://github.com/JinFree/Recycle_Classification_Dataset.git')
      shutil.rmtree(os.path.join(os.getcwd(), "Recycle_Classification_Dataset", ".git"))  # optinal
    self.USE_CUDA = torch.cuda.is_available()
    self.DEVICE = torch.device('cuda' if self.USE_CUDA else 'cpu')
    self.transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    dataset = Dataset(dataset_dir, self.transform)
    dataset.__save_label_map__()
    self.num_classes = dataset.__num_classes__()
    train_size = int(len(dataset)*train_ratio)
    test_size = len(dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    self.train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    self.test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )
    self.model = None
    self.model_str = None

  def prepare_network(self, is_scratch=True):
    if is_scratch:
      self.model = MODEL_From_Scratch(self.num_classes)
      self.model_str = "PyTorch_Training_From_Scratch"
    else:
      self.model = MobileNet(self.num_classes)
      self.model_str = "PyTorch_Transfer_Learning_MobileNet"
    self.model.to(self.DEVICE)
    self.model_str += ".pt"

  def training_network(self, learning_rate=0.001, epochs=10,
                       step_size=3, gamma=0.3):
    if self.model is None:
      self.prepare_network(False)
    optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size,
                                          gamma=gamma)
    acc = 0.0
    for epoch in range(epochs):
      print(f'Use CUDA = {torch.cuda.is_available()}')
      self.model.train()
      for data, target in tqdm(self.train_loader):
        data, target = data.to(self.DEVICE), target.to(self.DEVICE)
        output = self.model(data)
        loss = F.cross_entropy(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      scheduler.step()
      self.model.eval()
      test_loss = 0
      correct = 0

      with torch.no_grad():
        for data, target in tqdm(self.test_loader):
          data, target = data.to(self.DEVICE), target.to(self.DEVICE)
          output = self.model(data)
          test_loss += F.cross_entropy(output, target).item()
          pred = output.argmax(-1)
          correct += (pred == target).float().mean().item()

      test_loss /= len(self.test_loader)
      test_accuracy = correct / len(self.test_loader)
      print(f"epoch = {epoch}, test loss = {test_loss:.3f}, test acc = {test_accuracy:.3f}")
      if acc < test_accuracy:
        acc = test_accuracy
        torch.save(self.model.state_dict(), self.model_str)
        print('model saved!')

if __name__ == '__main__':
  training_class = PyTorch_Classification_Training_Class()
  # training_class.prepare_network(True)  # Scratch_model
  training_class.prepare_network(False) # MobileNet
  training_class.training_network(learning_rate=0.001, epochs=10, step_size=3, gamma=0.3)

Overwriting PyTorch_Classification_Training_Class.py


In [ ]:
!python PyTorch_Classification_Training_Class.py

# Inference

In [9]:
%%writefile Inference_Cam.py

import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from Model_Class_From_the_Scratch import MODEL_From_Scratch
from Model_Class_Transfer_Learning_MobileNet import MobileNet

class Inference_Class():
  def __init__(self):
    USE_CUDA = torch.cuda.is_available()
    self.DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
    self.model = None
    self.label_map = None
    self.transform_info = transforms.Compose(
        [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )

  def load_model(self, is_train_from_scratch, label_map_text='label_map.txt'):
    self.label_map = np.loadtxt(label_map_text, str, delimiter="\n")
    num_classes = len(self.label_map)
    model_str = None
    if is_train_from_scratch:
      self.model = MODEL_From_Scratch(num_classes).to(self.DEVICE)
      model_str = "PyTorch_Training_From_Scratch"
    else:
      self.model = MobileNet(num_classes).to(self.DEVICE)
      model_str = "PyTorch_Transfer_Learning_MobileNet"
    model_str = model_str + ".pt"

    self.model.load_state_dict(torch.load(model_str, map_location=self.DEVICE))
    self.model.eval()

Overwriting Inference_Cam.py


In [ ]:
from Inference_Cam import Inference_Class

inferenceClass = Inference_Class()
is_train_from_scratch = True
inferenceClass.load_model(is_train_from_scratch)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def inference(input_image):
  cv_image = []
  if isinstance(input_image, str):
    cv_image = cv2.imread(input_image, cv2.IMREAD_COLOR)
  else:
    cv_image = np.copy(input_image)

  result_frame, label_text, class_prob = inferenceClass.inference_image(cv_image)